# CNN(Convolutional Neural Network)
- Used framework: pytorch

In [21]:
import numpy as np 
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [22]:
seed = 1

batch_size = 64
test_batch_size = 64

no_cuda = False

In [23]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

## Data Preprocessing

In [24]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize(mean=(0.1307,), std=(0.3081,))
                    ])),
    batch_size=batch_size,
    shuffle=True)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize(mean=(0.1307,), std=(0.3081,))
                    ])),
    batch_size=test_batch_size,
    shuffle=True)

In [25]:
image, label = next(iter(train_loader))
image.shape, label.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

## Model

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        # Feature Extaction
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)

        # Classification
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Optimization

In [27]:
model = Net().to(device)

In [28]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

## Training & Evaluation

In [32]:
epochs = 10
log_interval = 100

In [36]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {: .6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()
            ))

    # Evaluation Mode
    model.eval()

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)
    ))

Train Epoch: 1 [0/60000 (0%)]	Loss:  0.014684
Train Epoch: 1 [6400/60000 (11%)]	Loss:  0.048980
Train Epoch: 1 [12800/60000 (21%)]	Loss:  0.036658
Train Epoch: 1 [19200/60000 (32%)]	Loss:  0.015822
Train Epoch: 1 [25600/60000 (43%)]	Loss:  0.016382
Train Epoch: 1 [32000/60000 (53%)]	Loss:  0.178071
Train Epoch: 1 [38400/60000 (64%)]	Loss:  0.058943
Train Epoch: 1 [44800/60000 (75%)]	Loss:  0.008323
Train Epoch: 1 [51200/60000 (85%)]	Loss:  0.074407
Train Epoch: 1 [57600/60000 (96%)]	Loss:  0.037734

Test set: Average Loss: 0.0548, Accuracy: 9835/10000 (98.3500%)

Train Epoch: 2 [0/60000 (0%)]	Loss:  0.088294
Train Epoch: 2 [6400/60000 (11%)]	Loss:  0.157994
Train Epoch: 2 [12800/60000 (21%)]	Loss:  0.019574
Train Epoch: 2 [19200/60000 (32%)]	Loss:  0.027723
Train Epoch: 2 [25600/60000 (43%)]	Loss:  0.073723
Train Epoch: 2 [32000/60000 (53%)]	Loss:  0.113683
Train Epoch: 2 [38400/60000 (64%)]	Loss:  0.114646
Train Epoch: 2 [44800/60000 (75%)]	Loss:  0.017126
Train Epoch: 2 [51200/60000 